In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
offers = portfolio.copy()

In [3]:
# Collecting transactions from transcript
transactions = transcript.copy()

In [4]:
# Opening value field to get amount spent to an own column
transactions_opened = pd.concat([transactions.drop(['value'], axis=1), transactions['value'].apply(pd.Series)], axis=1)

In [5]:
transactions = transactions_opened.copy()

In [6]:
# It seems there might be offer ids in two columns, 'offer id' and 'offer_id'. 
## Combining those if that's the case.
def combine_offer_id_cols(data):
    data['offer id'] = data['offer id'].fillna(data['offer_id'])
    data.drop(['offer_id'], inplace=True, axis=1)
    return data

combine_offer_id_cols(transactions)

transactions.head(10)

,event,person,time,offer id,amount,reward
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
5,offer received,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
6,offer received,c4863c7985cf408faee930f111475da3,0,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN
7,offer received,2eeac8d8feae4a8cad5a6af0499a211d,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN
8,offer received,aa4862eba776480b8bb9c68455b8c2e1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
9,offer received,31dda685af34476cad5bc968bdb01c53,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN


In [7]:
transactions = transactions.sort_values(by=['time', 'offer id'])

In [8]:
transactions['offer_duration'] = transactions['offer id'].map(offers.set_index('id')['duration'])

In [9]:
transactions

,event,person,time,offer id,amount,reward,offer_duration
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
8,offer received,aa4862eba776480b8bb9c68455b8c2e1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
9,offer received,31dda685af34476cad5bc968bdb01c53,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
10,offer received,744d603ef08c4f33af5a61c8c7628d1c,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
11,offer received,3d02345581554e81b7b289ab5e288078,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
14,offer received,d53717f5400c4e84affdaeda9dd926b3,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
16,offer received,d058f73bf8674a26a95227db098147b1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
68,offer received,3f4e408b0b2d4ea89dd16d7437d867c1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
101,offer received,88baa20c29a94178a43a7d68e5f039d4,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
102,offer received,23f9e3baf42b43148fee9bbbc3530c7a,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0


In [10]:
# We cannot count without offer id, so leaving them for a while

offer_actions = transactions[transactions['event'] != 'transaction']
offer_actions

,event,person,time,offer id,amount,reward,offer_duration
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
8,offer received,aa4862eba776480b8bb9c68455b8c2e1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
9,offer received,31dda685af34476cad5bc968bdb01c53,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
10,offer received,744d603ef08c4f33af5a61c8c7628d1c,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
11,offer received,3d02345581554e81b7b289ab5e288078,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
14,offer received,d53717f5400c4e84affdaeda9dd926b3,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
16,offer received,d058f73bf8674a26a95227db098147b1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
68,offer received,3f4e408b0b2d4ea89dd16d7437d867c1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
101,offer received,88baa20c29a94178a43a7d68e5f039d4,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0
102,offer received,23f9e3baf42b43148fee9bbbc3530c7a,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0


In [11]:
transactions['unique_offer_cum'] = offer_actions.groupby(['person', 'offer id', 'event']).cumcount()

In [12]:
transactions[transactions['unique_offer_cum']>1].head()

,event,person,time,offer id,amount,reward,offer_duration,unique_offer_cum
111932,offer received,158937e9d76e4ca7923a8b5499564f5d,336,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,2.0
112066,offer received,879be27695d74227bc9974662ec74852,336,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,2.0
113370,offer received,8bdaca43bc4b4446ad8e86a82489fd23,336,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,2.0
117789,offer received,8ed66913c59348ca9e5ff003863ff830,336,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,2.0
118032,offer received,43a50d7e5c8547a9aa1ec31291b9fc30,336,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,2.0


In [13]:
transactions = pd.concat([transactions, 
                    pd.get_dummies(transactions['event'], 
                    prefix=None)], 
                    axis=1)
transactions.drop(['event'], axis=1, inplace=True)

In [14]:
transactions.head()

,person,time,offer id,amount,reward,offer_duration,unique_offer_cum,offer completed,offer received,offer viewed,transaction
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,0.0,0,1,0,0
8,aa4862eba776480b8bb9c68455b8c2e1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,0.0,0,1,0,0
9,31dda685af34476cad5bc968bdb01c53,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,0.0,0,1,0,0
10,744d603ef08c4f33af5a61c8c7628d1c,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,0.0,0,1,0,0
11,3d02345581554e81b7b289ab5e288078,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,0.0,0,1,0,0


In [15]:
transactions.loc[transactions['offer received'] == 1, 'received time'] = transactions['time']
transactions.loc[transactions['offer viewed'] == 1, 'viewed time'] = transactions['time']
transactions.loc[transactions['offer completed'] == 1, 'completion time'] = transactions['time']

In [16]:
transactions.sort_values(by=['time'])

,person,time,offer id,amount,reward,offer_duration,unique_offer_cum,offer completed,offer received,offer viewed,transaction,received time,viewed time,completion time
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,10.0,0.0,0,1,0,0,0.0,NaN,NaN
2452,aaf27a405d4c4632b28f29ca0279c54a,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2466,f80eb9caef944ca899f2bef52b6c2f5e,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2468,b79e77ff80f443ff8aa51232799fd49b,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2480,e300729d05c544cfad882e049e867a70,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2490,ec09bad002ba40f1b327714e29cd50fd,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2493,82eafcbf423a47c49d111695f03d31b5,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2501,5faae0835a8049c1b917d6d34316cc87,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2507,44f1f503047642ac83f70785c4992032,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN
2519,9a6e41f3f689410bab74f638363ef95a,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,7.0,0.0,0,1,0,0,0.0,NaN,NaN


In [17]:
# select informational only

transactions_informational = transactions[(transactions['offer id'].isin(['5a8bc65990b245e5a138643cd4eb9837', '3f207df678b143eea3cee63160fa8bed']))]
transactions_informational

,person,time,offer id,amount,reward,offer_duration,unique_offer_cum,offer completed,offer received,offer viewed,transaction,received time,viewed time,completion time
7,2eeac8d8feae4a8cad5a6af0499a211d,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
13,c27e0d6ab72c455a8bb66d980963de60,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
56,2c5c63e5124e417b8504f76c868a4190,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
76,5e9c83e8ab4d4e9ca83f3256d83ca756,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
90,6747dda08bed46298c23e05b64820a3a,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
99,37a97f6e09784a07bbb67bc6539f1a00,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
100,65f06566a2a14f64b003964f211635e1,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
112,0f54094029ca4107bfd36203aeb3ed14,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
116,fd700397b7c8440baa70d5539926c5a0,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN
121,3ecb372184104ff7bad0f0f60cb56e29,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,4.0,0.0,0,1,0,0,0.0,NaN,NaN


In [18]:
aggregation_functions = {
    'offer received': 'sum', 
    'offer viewed': 'sum',
    'offer completed': 'sum', 
    'reward': 'sum',
    'offer_duration': 'first',
    'received time': 'first',
    'viewed time': 'first',
    'completion time': 'first'}

transactions_informational = transactions_informational.groupby(['person','offer id','unique_offer_cum']).aggregate(aggregation_functions)
transactions_informational.reset_index(inplace=True)

In [19]:
transactions_informational.head()

,person,offer id,unique_offer_cum,offer received,offer viewed,offer completed,reward,offer_duration,received time,viewed time,completion time
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,0.0,1,1,0,0.0,4.0,336.0,372.0,NaN
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,168.0,192.0,NaN
2,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,0.0,1,1,0,0.0,4.0,0.0,6.0,NaN
3,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,336.0,354.0,NaN
4,0020c2b971eb4e9188eac86d93036a77,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,504.0,660.0,NaN


In [20]:
# Create table with transactions
transactions_table = transactions[(transactions['transaction'] == 1)]
#transactions_table

In [21]:
test_data = transactions_informational[300:320].copy()
test_data

,person,offer id,unique_offer_cum,offer received,offer viewed,offer completed,reward,offer_duration,received time,viewed time,completion time
300,050a57f4d675402ba1a06c8c49a4a200,3f207df678b143eea3cee63160fa8bed,1.0,1,0,0,0.0,4.0,504.0,NaN,NaN
301,051006a0cddf40e0a973d9317c032086,3f207df678b143eea3cee63160fa8bed,0.0,1,0,0,0.0,4.0,336.0,NaN,NaN
302,051006a0cddf40e0a973d9317c032086,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,408.0,432.0,NaN
303,0517e2aca44d4327875c3bb8478e492e,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,336.0,522.0,NaN
304,0517e2aca44d4327875c3bb8478e492e,5a8bc65990b245e5a138643cd4eb9837,1.0,1,0,0,0.0,3.0,504.0,NaN,NaN
305,051b92f5a9f94328a8721e86905c1d3d,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,168.0,180.0,NaN
306,05298f5390c7417c8dbbcbef9f081800,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,576.0,588.0,NaN
307,0538071eb35247af8c2816ec75a3c87c,3f207df678b143eea3cee63160fa8bed,0.0,1,0,0,0.0,4.0,408.0,NaN,NaN
308,0538071eb35247af8c2816ec75a3c87c,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,504.0,504.0,NaN
309,053de26a38684d628b83fef38eae0c69,3f207df678b143eea3cee63160fa8bed,0.0,1,0,0,0.0,4.0,0.0,NaN,NaN


In [22]:
# list of customer ids having transactions so that it is easy to check if a person is there or not
customers_with_transactions = transactions_table.person.tolist()

In [32]:
def count_transactions(data):
    #print(data.person.isin(transactions_table.person))
    #print(data.person.any())    
    
    for index, row in data.iterrows():
                
        # not viewed
        if (str(row['viewed time'])=='nan'):
            
            # and no transactions
            if row['person'] not in customers_with_transactions:
                status = 'churn'
            # and transactions
            else:
                status = 'active'
        
        # viewed
        elif (row['viewed time'] >= 0):
        
            effect_until = row['received time'] + 24*row['offer_duration']
            
            # but too late
            if row['viewed time'] > effect_until:
                status = 'late'
            
            # but no transactions
            elif row['person'] not in customers_with_transactions:
                status = 'disinterested'
                
            # in time and has transactions
            else:
                cust_transactions_during_effect_time = transactions_table[
                    (transactions_table.person == row['person']) 
                    & (transactions_table.time <= effect_until) 
                    & (transactions_table.time > row['received time'])]
                
                # there's transaction(s) during the effective time
                if (len(cust_transactions_during_effect_time)>0):
                    status = 'effected'                
                else:
                    status = 'disinterested'
    
        data.at[index,'status'] = status
        
    return data
    
count_transactions(transactions_informational)

,person,offer id,unique_offer_cum,offer received,offer viewed,offer completed,reward,offer_duration,received time,viewed time,completion time,status
300,050a57f4d675402ba1a06c8c49a4a200,3f207df678b143eea3cee63160fa8bed,1.0,1,0,0,0.0,4.0,504.0,NaN,NaN,active
301,051006a0cddf40e0a973d9317c032086,3f207df678b143eea3cee63160fa8bed,0.0,1,0,0,0.0,4.0,336.0,NaN,NaN,active
302,051006a0cddf40e0a973d9317c032086,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,408.0,432.0,NaN,effected
303,0517e2aca44d4327875c3bb8478e492e,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,336.0,522.0,NaN,late
304,0517e2aca44d4327875c3bb8478e492e,5a8bc65990b245e5a138643cd4eb9837,1.0,1,0,0,0.0,3.0,504.0,NaN,NaN,active
305,051b92f5a9f94328a8721e86905c1d3d,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,168.0,180.0,NaN,disinterested
306,05298f5390c7417c8dbbcbef9f081800,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,576.0,588.0,NaN,effected
307,0538071eb35247af8c2816ec75a3c87c,3f207df678b143eea3cee63160fa8bed,0.0,1,0,0,0.0,4.0,408.0,NaN,NaN,active
308,0538071eb35247af8c2816ec75a3c87c,5a8bc65990b245e5a138643cd4eb9837,0.0,1,1,0,0.0,3.0,504.0,504.0,NaN,effected
309,053de26a38684d628b83fef38eae0c69,3f207df678b143eea3cee63160fa8bed,0.0,1,0,0,0.0,4.0,0.0,NaN,NaN,active


In [25]:
#transactions_informational[transactions_informational['status'] == 'churn']